In [ ]:
import json

with open('documents-with-idxs.json') as f_in:
    documents = json.load(f_in)

In [ ]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-MiniLM-L6-cos-v1"
model = SentenceTransformer(model_name)

In [ ]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

index_name = "course-question"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    doc['question_vector'] = model.encode(doc['question'])
    doc['text_vector'] = model.encode(doc['text'])
    doc['question_text_vector'] = model.encode(doc['question'] + ' ' + doc['text'])

    es_client.index(index=index_name, document=doc)

In [ ]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
def vector_knn(_dict, query_vector):
    question = _dict['question']
    course = _dict['course']

    q_v = model.encode(question)

    return elastic_search_knn(query_vector, q_v, course)

In [ ]:
import pandas as pd

df_ground_truth = pd.read_csv("ground-truth-data.csv")
ground_truth = df_ground_truth.to_dict(orient='records')

In [ ]:
def hit_rate(relevance_total):
    counter = 0

    for line in relevance_total:
        if True in line:
            counter = counter + 1

    return counter / len(relevance_total)


def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)


def evaluate(ground_truth, search_function, query_vector):
    print(query_vector)
    relevance_total = []

    for _dict in tqdm(ground_truth):
        doc_id = _dict['document']
        
        results = search_function(_dict, query_vector)
        relevance = [doc['id'] == doc_id for doc in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [ ]:
evaluate(ground_truth, vector_knn, "question_vector")

In [ ]:
evaluate(ground_truth, vector_knn, "text_vector")

In [ ]:
evaluate(ground_truth, vector_knn, "question_text_vector")

In [ ]:
def elastic_search_knn_combined(vector, course):
    ## kosinüs benzerliği [-1, 1] hesaplamalarına +1 eklenmesinin amacı,
        ## sonuçların her zaman pozitif bir değer almasını sağlamaktır.
    
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "course": course
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'text_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_text_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [ ]:
def evaluate_combined(ground_truth, search_function):
    relevance_total = []

    for _dict in tqdm(ground_truth):
        doc_id = _dict['document']
        
        results = search_function(_dict)
        relevance = [doc['id'] == doc_id for doc in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }


def vector_combined_knn(_dict):
    question = _dict['question']
    course = _dict['course']

    q_v = model.encode(question)

    return elastic_search_knn_combined(q_v, course)

evaluate_combined(ground_truth, vector_combined_knn)